In [1]:
%pip install --upgrade --quiet "google-cloud-aiplatform>=1.100.0" "google-adk>=1.5.0"


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Use the environment variable if the user doesn't provide Project ID.
import os
import uuid

import vertexai

# Project configuration
# @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
PROJECT_ID = "alpha-code-461805"

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# Set environment variables required for ADK
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

# Agent configuration
MODEL_NAME = "gemini-2.5-flash"
USER_ID = f"user_{uuid.uuid4()}"

print(f"Project: {PROJECT_ID}")
print(f"Location: {LOCATION}")
print(f"Session User ID: {USER_ID}")

# Initialize Vertex AI client
_ = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,)

Project: alpha-code-461805
Location: us-central1
Session User ID: user_ddbda307-270c-4d35-be03-07f1a7e24260


In [3]:
import uuid
import vertexai
from vertexai import agent_engines
from google import adk
from google.adk.memory import VertexAiMemoryBankService
from google.adk.sessions import VertexAiSessionService
from google.genai import types

In [4]:
def run_single_turn(query, session, user_id):
    """Run a single conversation turn."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session,
                        new_message=content)

    response_content = None
    for event in events:
        if event.is_final_response():
            response_content = event.content.parts[0].text
    return response_content


def chat_loop(session, user_id) -> None:
    """Main chat interface loop."""
    print("\nStarting chat. Type 'exit' or 'quit' to end.")
    print("Every message will be automatically stored in memory.\n")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["quit", "exit", "bye"]:
            print("\nAssistant: Thank you for chatting. Have a great day!")
            break

        response = run_single_turn(user_input, session, user_id)
        if response:
            print(f"\nAssistant: {response}")

In [5]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=fYQYoftzHhjNUetFBZugKFbgBbmtj8&access_type=offline&code_challenge=gjmBU0MNtUW7bkmrtLF_XZ1TUi40In6fuC1eaISqG50&code_challenge_method=S256


Credentials saved to file: [/Users/ashwinikm/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "alpha-code-461805" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [11]:
# agent_engine = agent_engines.create(display_name="Scrapping Agent Engine",description="An agent engine for scrapping and summarizing URLs")
# print(f"Created Agent Engine: {agent_engine.resource_name}")

agent_engine = agent_engines.get('projects/alpha-code-461805/locations/us-central1/reasoningEngines/1583745344741572608')
print(f"Retrieved Agent Engine: {agent_engine.resource_name}")

Retrieved Agent Engine: projects/alpha-code-461805/locations/us-central1/reasoningEngines/1583745344741572608


In [12]:
# agent.py
import os
from a2a.client import A2AClient, A2ACardResolver
import logging
from uuid import uuid4
from google.adk.tools.function_tool import FunctionTool
from google.genai import types
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.agents.llm_agent import LlmAgent
import httpx
from dotenv import load_dotenv
from a2a.types import (
    AgentCard
)
import logging
from google.adk.tools.preload_memory_tool import PreloadMemoryTool
from a2a.types import (
    AgentCard,
    SendMessageRequest,
)

load_dotenv()
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)


# Retrieve the Google model name from environment variables with a default fallback.
model_name = os.getenv("GOOGLE_MODEL_NAME", "gemini-2.5-pro")

# Retrieve the A2A agent registry base URL from environment variables with a default fallback.
AGENT_REGISTRY_BASE_URL = os.getenv(
    "AGENT_REGISTRY_BASE_URL", "https://search-agent-100889782425.us-central1.run.app/")

# Extracted list_agents function


async def list_agents() -> list[dict]:
    """
    Fetch all AgentCard metadata from the registry,
    return as a list of plain dicts.
    """
    async with httpx.AsyncClient() as httpx_client:
        base_url = AGENT_REGISTRY_BASE_URL.rstrip("/")
        # response = await client.get(url, timeout=50.0)

        logger.info("Initializing A2ACardResolver to fetch agent capabilities.")
        resolver = A2ACardResolver(
            httpx_client=httpx.AsyncClient(timeout=3000),
            base_url=base_url,
            # agent_card_path and extended_agent_card_path use defaults if not specified
        )
        final_agent_card_to_use: AgentCard | None = None

        try:
            logger.info(
                f"Attempting to fetch public agent card from: {base_url}"
            )
            # Fetches the AgentCard from the standard public path.
            public_card = await resolver.get_agent_card()
            logger.info("Successfully fetched public agent card:")
            logger.info(
                public_card.model_dump_json(indent=2, exclude_none=True)
            )
            final_agent_card_to_use = public_card
            logger.info(
                "Using PUBLIC agent card for A2AClient initialization.")

        except Exception as e:
            logger.error(
                f"Critical error fetching public agent card from {base_url}: {e}",
                exc_info=True  # This prints the full traceback, very helpful for debugging
            )
            raise RuntimeError(
                "Failed to fetch the public agent card. Cannot continue."
            ) from e

        cards_data = final_agent_card_to_use
        # Assuming AgentCard has a 'name' attribute, print the name instead of length
        print(f"Fetched agent '{cards_data.name}' from registry at {base_url}")
        print(f"Agent data: {cards_data}")
        return cards_data

# Extracted call_agent function


async def call_agent(agent_name: str, message: str) -> str:
    """
    Given an agent_name string and a user message,
    find that agent's URL, send the task, and return its reply.
    """
    cards = await list_agents()  # Use the module-level list_agents

    client = A2AClient(httpx_client=httpx.AsyncClient(timeout=3000),
                       agent_card=cards)

    print("Connected to A2AClient at", AGENT_REGISTRY_BASE_URL)
    session_id = "transalation_session"
    # Find the agent card by name
    task_id = uuid4().hex

    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [
                    {
                        "kind": "text",
                        "text": message
                    }
                ],
                "messageId": uuid4().hex
            },
            "metadata": {}
        }
    }

    # Using user_id as session_id for simplicity as in original code

    response_rec = await client.send_message(SendMessageRequest(**payload))
    # print("Response from send_message:", reasponse_rec.model_dump(
    #     mode='json', exclude_none=True))

    print("Response from send_message:", response_rec)

    response = response_rec.model_dump(mode='json', exclude_none=True)
    # print("Formatted Response:", response)
    # print("parse", response['result']['status']['message']['parts'][0]['text'])
    return (response['result']['status']['message']['parts'][0]['text'])

# System instruction for the LLM
system_instr = (
    "You are a root orchestrator agent. You have three tools:\n"
    "1) list_agents() → Use this tool to see a list of all available agents and their capabilities.\n"
    "2) call_agent(agent_name: str, message: str) → Use this tool to send a message to a specific agent by its name and get its response.\n"
    "3) memory → Use this tool to preload memory with relevant information and answer if user ask some personal questions\n"
    "Use these tools to fulfill user requests by discovering and interacting with other agents as needed.\n"
)
memory = PreloadMemoryTool()

# Create the LlmAgent instance directly
agent = adk.Agent(
    model=model_name,
    name="root_orchestrator",
    description="Discovers and orchestrates other agents",
    instruction=system_instr,
    tools=[
        memory,
        FunctionTool(list_agents),
        FunctionTool(call_agent),
    ],
)

In [13]:
app_name = "url_scrape_agent_" + str(uuid.uuid4())[:6]
agent_engine_id = agent_engine.name

memory_bank_service = VertexAiMemoryBankService(
    project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
)


session_service = VertexAiSessionService(
    project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
)

runner = adk.Runner(
    agent=agent,
    app_name=app_name,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [14]:
session1 = await runner.session_service.create_session(
    app_name=app_name,
    user_id=USER_ID,
)

2025-07-18 10:11:25,151 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:11:27,035 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:11:27,281 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:11:27,288 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:11:27,308 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x153951070>
2025-07-18 10:11:27,311 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x1532a15d0> server_hostname='us-central1-aiplatform.googleapis.com' timeout=None
2025-07-18 10:11:27,351 - DEBUG - httpcore.connection 

In [16]:
chat_loop(session1.id, USER_ID)


Starting chat. Type 'exit' or 'quit' to end.
Every message will be automatically stored in memory.



2025-07-18 10:12:23,078 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-18 10:12:23,079 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:12:23,617 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:12:23,808 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:12:23,810 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:12:23,830 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x153995a60>
2025-07-18 10:12:23,830 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x15411fbd0> server_hostname='us-central1-aiplatform.googleap


Assistant: Hello Ashwini, it's a pleasure to meet you! I'd be happy to help you with any travel-related questions you may have. What can I do for you today?


2025-07-18 10:13:18,019 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-18 10:13:18,020 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:13:18,746 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:13:18,974 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:13:18,975 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:13:18,992 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x15399f860>
2025-07-18 10:13:18,993 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x154107250> server_hostname='us-central1-aiplatform.googleap


Assistant: That sounds wonderful! Paris is an amazing city with a rich history and incredible cuisine. What was your favorite food you tried there?


2025-07-18 10:15:19,352 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-18 10:15:19,353 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:15:19,853 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:15:20,079 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:15:20,082 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:15:20,105 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x153996780>
2025-07-18 10:15:20,105 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x15411d0d0> server_hostname='us-central1-aiplatform.googleap


Assistant: Virat Kohli is a fantastic player! His dedication and skill are truly impressive.

Assistant: Thank you for chatting. Have a great day!


In [17]:
completed_session = await runner.session_service.get_session(
    app_name=app_name, user_id=USER_ID, session_id=session1.id
)
await memory_bank_service.add_session_to_memory(completed_session)

2025-07-18 10:17:30,188 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:17:30,725 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:17:30,899 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:17:30,901 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:17:31,076 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x15399e840>
2025-07-18 10:17:31,077 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x153c57550> server_hostname='us-central1-aiplatform.googleapis.com' timeout=None
2025-07-18 10:17:31,119 - DEBUG - httpcore.connection 

2025-07-18 10:17:36,855 - DEBUG - httpcore.connection - close.complete


In [18]:
session2 = await runner.session_service.create_session(
    app_name=agent_engine.name,
    user_id=USER_ID,
)

2025-07-18 10:18:23,876 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:18:24,493 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:18:24,666 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:18:24,667 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:18:24,682 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x15368ac00>
2025-07-18 10:18:24,683 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x1536439d0> server_hostname='us-central1-aiplatform.googleapis.com' timeout=None
2025-07-18 10:18:24,723 - DEBUG - httpcore.connection 

In [19]:
chat_loop(session2.id, USER_ID)


Starting chat. Type 'exit' or 'quit' to end.
Every message will be automatically stored in memory.



2025-07-18 10:18:40,999 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-18 10:18:41,000 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:18:41,367 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:18:41,562 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:18:41,564 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:18:41,582 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x15394ca10>
2025-07-18 10:18:41,583 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x1536405d0> server_hostname='us-central1-aiplatform.googleap


Assistant: Based on our previous conversations, I know that your name is Ashwini, your favorite player is Virat Kohli, and you love to travel.


2025-07-18 10:19:14,408 - DEBUG - asyncio - Using selector: KqueueSelector
2025-07-18 10:19:14,409 - INFO - google_genai._api_client - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.
2025-07-18 10:19:14,746 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-07-18 10:19:14,962 - DEBUG - urllib3.connectionpool - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-07-18 10:19:14,965 - DEBUG - httpcore.connection - connect_tcp.started host='us-central1-aiplatform.googleapis.com' port=443 local_address=None timeout=None socket_options=None
2025-07-18 10:19:14,986 - DEBUG - httpcore.connection - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x1539a0260>
2025-07-18 10:19:14,986 - DEBUG - httpcore.connection - start_tls.started ssl_context=<ssl.SSLContext object at 0x153643d50> server_hostname='us-central1-aiplatform.googleap


Assistant: You were in Paris last year.

Assistant: Thank you for chatting. Have a great day!
